In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import os

def download_images(query, max_images, save_dir):
    # Set up Selenium WebDriver
    driver = webdriver.Chrome()  # Use the path to your ChromeDriver if necessary
    driver.get(f"https://www.google.com/search?tbm=isch&q={query}")

    # Create the directory to save images
    os.makedirs(save_dir, exist_ok=True)

    # Scroll and collect image URLs
    image_urls = set()
    while len(image_urls) < max_images:
        thumbnails = driver.find_elements(By.CSS_SELECTOR, "img.rg_i")
        for img in thumbnails[len(image_urls):]:
            try:
                img.click()
                time.sleep(2)
                larger_image = driver.find_element(By.CSS_SELECTOR, "img.n3VNCb")
                src = larger_image.get_attribute("src")
                if src.startswith("http") and len(image_urls) < max_images:
                    image_urls.add(src)
            except Exception as e:
                print(f"Error clicking image: {e}")

        # Scroll down to load more images
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # Download images
    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url)
            with open(os.path.join(save_dir, f"{query}_{i + 1}.jpg"), "wb") as file:
                file.write(response.content)
            print(f"Downloaded: {query}_{i + 1}.jpg")
        except Exception as e:
            print(f"Could not download {url}: {e}")

    driver.quit()

# Example usage
download_images("puppies", max_images=10, save_dir="images")




In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

MIN_MATCH_COUNT = 30

sift = cv2.SIFT_create()

FLANN_INDEX_KDTREE = 0
flann_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
flann = cv2.FlannBasedMatcher(flann_params, {})

dataset_path = "path/to/positive_samples"  

train_images = []
kp_list = []
desc_list = []

for filename in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, filename)
    train_img = cv2.imread(img_path, 0)
    if train_img is not None:
        kp, desc = sift.detectAndCompute(train_img, None)
        if desc is not None:
            train_images.append(train_img)
            kp_list.append(kp)
            desc_list.append(desc)

print(f"Loaded {len(train_images)} training images.")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp2, desc2 = sift.detectAndCompute(gray, None)
    if desc2 is None:
        continue

    best_match_count = 0
    best_match_outline = None

    for i in range(len(train_images)):
        matches = flann.knnMatch(desc2, desc_list[i], k=2)
        good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
        
        if len(good_matches) > best_match_count:
            best_match_count = len(good_matches)
            tp = np.float32([kp_list[i][m.trainIdx].pt for m in good_matches])
            qp = np.float32([kp2[m.queryIdx].pt for m in good_matches])
            H, status = cv2.findHomography(tp, qp, cv2.RANSAC, 3.0)
            
            if H is not None:
                h, w = train_images[i].shape
                train_outline = np.float32([[[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]])
                best_match_outline = cv2.perspectiveTransform(train_outline, H)
    if best_match_count > MIN_MATCH_COUNT and best_match_outline is not None:
        cv2.polylines(frame, [np.int32(best_match_outline)], True, (0, 255, 0), 3)
        cv2.putText(frame, 'Object Detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        print(f"Match Found: {best_match_count} keypoints")
    else:
        print("No strong match found")
    
    # Display the frame
    cv2.imshow('Object Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == 27: 
        break

cap.release()
cv2.destroyAllWindows()
